<a href="https://colab.research.google.com/github/MengOonLee/BertelsmannAITrack/blob/MyLoveSong/LoveSongGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%bash

wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-B1H4krto65n8hA3Qmm2UZ0_TZO2JE3t' -qO LoveSongs.txt